In [1]:
!pip install transformers datasets torch evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00


In [2]:
import evaluate
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np

## DistilBERT on GLUE Tasks

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [ ]:
task_to_keys = {
    "sst2": ("sentence", None),
    "cola": ("sentence", None),
    "mrpc": ("sentence1", "sentence2"),
    "stsb": ("sentence1", "sentence2"),
    "qqp": ("question1", "question2"),
    "qnli": ("question", "sentence"),
    "mnli": ("premise", "hypothesis"),
    "rte": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [ ]:
def train_and_evaluate(task):

    # Step 1: Load the dataset
    dataset = load_dataset("glue", task)

    num_labels = 3 if task == "mnli" else 1 if task =="stsb" else 2
    batch_size = 32 if task in ["qqp", "mnli", "qnli"] else 16
    max_length = 128

    # Step 2: Load the tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    sentence1_key, sentence2_key = task_to_keys[task]

    # Step 3: Preprocess the dataset
    def preprocess_function(examples):
        if sentence2_key is None:
            return tokenizer(examples[sentence1_key], truncation=True, padding="max_length", max_length=max_length)
        return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, padding="max_length", max_length=max_length)

    tokenized_dataset = dataset.map(preprocess_function, batched=True)

    # Step 4: Load pre-trained model with a classification head
    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

    # Step 6: Load metric using the `evaluate` library
    metric = evaluate.load("glue", task)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        # Convert logits to a PyTorch tensor
        logits = torch.tensor(logits)
        predictions = logits[:, 0] if task == "stsb" else torch.argmax(logits, dim=-1)
        return metric.compute(predictions=predictions, references=labels)

    # Define a list of seeds for 5 different runs
    seeds = [42, 1234, 100, 456, 789]

    # Store results from each run
    all_results = []

    # Loop for 5 runs with different seeds
    for seed in seeds:
        print(f"Running with seed: {seed}")

        # Step 5: Define training arguments (update with the current seed)
        training_args = TrainingArguments(
          output_dir=f"./results_{task}",
          eval_strategy="epoch",
          save_strategy = "epoch",
          learning_rate=5e-5,
          per_device_train_batch_size=batch_size,
          per_device_eval_batch_size=batch_size,
          num_train_epochs=3,
          warmup_ratio=0.1,
          weight_decay=0.01,
          logging_dir=f"./logs_{task}_seed_{seed}",
          logging_steps=100,
          gradient_accumulation_steps=1,
          max_grad_norm=1.0,
          save_total_limit=2,
          fp16=True,
          seed=seed,
          load_best_model_at_end=True,
          report_to=[],
        )

        validation_key = "validation_matched" if task == "mnli" else "validation"
        # Step 7: Set up the Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset[validation_key],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
          )

        # Step 8: Train the model
        trainer.train()

        # Evaluate the model and store results
        results = trainer.evaluate()
        print(f"Results for {task.capitalize()} with seed {seed}: {results}")

        # Get the appropriate metric key based on the task
        metric_key = "eval_pearson" if task == "stsb" else "eval_matthews_correlation" if task == "cola" else "eval_accuracy"

        # Append the accuracy to the results list
        all_results.append(results[metric_key])

    # Calculate the median accuracy from all runs
    median_accuracy = np.median(all_results)

    # Print the final median result
    print(f"DistilBERT results are the medians of 5 runs with different seeds: {median_accuracy}")

In [ ]:
# Loop through each task and run training/evaluation
for task in GLUE_TASKS:
    print(f"Training and evaluating {task.upper()}...")
    train_and_evaluate(task)


Training and evaluating COLA...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.691193,0.563380
2,No log,0.699370,0.521127
3,0.696100,0.704879,0.352113


Results for Wnli with seed 456: {'eval_loss': 0.6911929845809937, 'eval_accuracy': 0.5633802816901409, 'eval_runtime': 0.0782, 'eval_samples_per_second': 908.165, 'eval_steps_per_second': 63.955, 'epoch': 3.0}
Running with seed: 789


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.708757,0.352113
2,No log,0.714610,0.295775
3,0.692800,0.724107,0.154930


Results for Wnli with seed 789: {'eval_loss': 0.7087572813034058, 'eval_accuracy': 0.352112676056338, 'eval_runtime': 0.1396, 'eval_samples_per_second': 508.731, 'eval_steps_per_second': 35.826, 'epoch': 3.0}
DistilBERT results are the medians of 5 runs with different seeds: 0.5633802816901409


## BERT on GLUE Tasks

In [ ]:
def train_and_evaluate_bert(task):

    # Step 1: Load the dataset
    dataset = load_dataset("glue", task)

    num_labels = 3 if task == "mnli" else 1 if task =="stsb" else 2
    batch_size = 32 if task in ["qqp", "mnli", "qnli"] else 16
    max_length = 128

    # Step 2: Load the tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    sentence1_key, sentence2_key = task_to_keys[task]

    # Step 3: Preprocess the dataset
    def preprocess_function(examples):
        if sentence2_key is None:
            return tokenizer(examples[sentence1_key], truncation=True, padding="max_length", max_length=max_length)
        return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, padding="max_length", max_length=max_length)

    tokenized_dataset = dataset.map(preprocess_function, batched=True)

    # Step 4: Load pre-trained model with a classification head
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

    # Step 6: Load metric using the `evaluate` library
    metric = evaluate.load("glue", task)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        # Convert logits to a PyTorch tensor
        logits = torch.tensor(logits)
        predictions = logits[:, 0] if task == "stsb" else torch.argmax(logits, dim=-1)
        return metric.compute(predictions=predictions, references=labels)

    # Define a list of seeds for 5 different runs
    seeds = [42, 1234, 100, 456, 789]

    # Store results from each run
    all_results = []

    # Loop for 5 runs with different seeds
    for seed in seeds:
        print(f"Running with seed: {seed}")

        # Step 5: Define training arguments (update with the current seed)
        training_args = TrainingArguments(
          output_dir=f"./bert_results_{task}",
          eval_strategy="epoch",
          save_strategy = "epoch",
          learning_rate=5e-6,
          per_device_train_batch_size=batch_size,
          per_device_eval_batch_size=batch_size,
          num_train_epochs=3,
          warmup_ratio=0.1,
          weight_decay=0.01,
          logging_dir=f"./bert_logs_{task}_seed_{seed}",
          logging_steps=100,
          # gradient_accumulation_steps=1,
          max_grad_norm=1.0,
          # save_total_limit=2,
          fp16=True,
          seed=seed,
          load_best_model_at_end=True,
          report_to=[],
        )

        validation_key = "validation_matched" if task == "mnli" else "validation"
        # Step 7: Set up the Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset[validation_key],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
          )

        # Step 8: Train the model
        trainer.train()

        # Evaluate the model and store results
        results = trainer.evaluate()
        print(f"Results for {task.capitalize()} with seed {seed}: {results}")

        metric_key = "eval_pearson" if task == "stsb" else "eval_matthews_correlation" if task == "cola" else "eval_accuracy"

        # Append the accuracy to the results list
        all_results.append(results[metric_key])

    # Calculate the median accuracy from all runs
    median_accuracy = np.median(all_results)

    # Print the final median result
    print(f"BERT results are the medians of 5 runs with different seeds: {median_accuracy}")

In [ ]:
# Loop through each task and run training/evaluation
for task in GLUE_TASKS:
    print(f"Training and evaluating {task.upper()}...")
    train_and_evaluate_bert(task)

Training and evaluating COLA...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Running with seed: 42


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## DistilBERT on IMDb Dataset

In [ ]:
# Import necessary libraries
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

# Load IMDb dataset and DistilBERT tokenizer
dataset = load_dataset("imdb")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", force_download=True)

# Tokenize data
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

train_data = dataset["train"].map(tokenize, batched=True)
test_data = dataset["test"].map(tokenize, batched=True)

# Convert labels to tensor format
train_data = train_data.with_format("torch", columns=["input_ids", "attention_mask", "label"])
test_data = test_data.with_format("torch", columns=["input_ids", "attention_mask", "label"])

# Load DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=[],
    logging_dir="./logs",
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes and returns a dictionary of metrics."""
    logits, labels = eval_pred
    logits = torch.tensor(logits)
    predictions = torch.argmax(logits, dim=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize Trainer with IMDb data
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)


# Train model
trainer.train()

# Evaluate model
results = trainer.evaluate()
print("IMDb Test Accuracy:", results["eval_accuracy"])


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.226200,0.237228,0.906080
2,0.154400,0.237441,0.929520
3,0.093400,0.281255,0.930600


IMDb Test Accuracy: 0.9306


## BERT on IMDb Dataset

In [ ]:
# Import necessary libraries
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load IMDb dataset and DistilBERT tokenizer
dataset = load_dataset("imdb")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", force_download=True)

# Tokenize data
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

train_data = dataset["train"].map(tokenize, batched=True)
test_data = dataset["test"].map(tokenize, batched=True)

# Convert labels to tensor format
train_data = train_data.with_format("torch", columns=["input_ids", "attention_mask", "label"])
test_data = test_data.with_format("torch", columns=["input_ids", "attention_mask", "label"])

# Load DistilBERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=[],
    logging_dir="./logs",
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes and returns a dictionary of metrics."""
    logits, labels = eval_pred
    logits = torch.tensor(logits)
    predictions = torch.argmax(logits, dim=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize Trainer with IMDb data
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

# Evaluate model
results = trainer.evaluate()
print("IMDb Test Accuracy:", results["eval_accuracy"])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered inter

Epoch,Training Loss,Validation Loss,Accuracy
1,0.206400,0.204074,0.922960
2,0.135000,0.214014,0.939360
3,0.079100,0.252385,0.942440


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered i

IMDb Test Accuracy: 0.94244


## DistilBERT on SQuAD Dataset (Without Distillation)

In [ ]:
# Necessary imports remain the same
from transformers import AutoTokenizer, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import evaluate

# Load SQuAD 1.1 dataset
squad = load_dataset("squad")

# Initialize evaluation metric
metric = evaluate.load("squad")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for i, offsets in enumerate(offset_mapping):
        answer = examples["answers"][i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        # Find the start and end token positions
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offsets[context_start][0] > start_char or offsets[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Apply preprocessing
encoded_dataset = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

# Function to compute metrics for SQuAD
# Function to compute metrics for SQuAD
def compute_metrics(eval_pred):
    """Computes and returns a dictionary of metrics."""
    start_logits, end_logits = eval_pred.predictions
    start_positions, end_positions = eval_pred.label_ids

    predictions = []
    references = []

    for i in range(len(start_logits)):
        start_pred = torch.argmax(torch.tensor(start_logits[i])).item()
        end_pred = torch.argmax(torch.tensor(end_logits[i])).item()

        # Retrieve input_ids and only pass those for decoding
        input_ids = encoded_dataset["validation"][i]["input_ids"]
        prediction_text = tokenizer.decode(input_ids[start_pred:end_pred+1], skip_special_tokens=True)

        predictions.append({"id": squad["validation"][i]["id"], "prediction_text": prediction_text})
        references.append({"id": squad["validation"][i]["id"], "answers": squad["validation"][i]["answers"]})

    return metric.compute(predictions=predictions, references=references)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="no",
    logging_steps=10,
    eval_steps=10,
    report_to="none"
)

# Load DistilBERT model
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train and evaluate the model
trainer.train()
results = trainer.evaluate()
print("DistilBERT Results without distillation on SQuAD:", results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact Match,F1
1,1.125800,1.137238,66.073794,76.175304
2,0.948700,1.093961,67.672658,77.820319
3,0.536200,1.191063,68.401135,78.217015


DistilBERT Results without distillation on SQuAD: {'eval_loss': 1.1910629272460938, 'eval_exact_match': 68.40113528855251, 'eval_f1': 78.21701520918465, 'eval_runtime': 43.458, 'eval_samples_per_second': 243.223, 'eval_steps_per_second': 15.21, 'epoch': 3.0}


## DistilBERT on SQuAD Dataset (With Distillation)

In [ ]:
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast, Trainer, TrainingArguments, BertForQuestionAnswering
from datasets import load_dataset
import evaluate
import numpy as np
import torch

# Load the SQuAD dataset
dataset = load_dataset("squad")

# Load DistilBERT model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

# Tokenize the data
def prepare_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=False,  # Disable overflow to avoid out-of-bounds
        return_offsets_mapping=True,
        padding="max_length",
    )

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        if len(examples["answers"][i]["answer_start"]) == 0:
            # No answer
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = examples["answers"][i]["answer_start"][0]
            end_char = start_char + len(examples["answers"][i]["text"][0])

            token_start_index, token_end_index = 0, 0
            for idx, (offset_start, offset_end) in enumerate(offsets):
                if offset_start <= start_char < offset_end:
                    token_start_index = idx
                if offset_start < end_char <= offset_end:
                    token_end_index = idx
                    break

            start_positions.append(token_start_index)
            end_positions.append(token_end_index)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

tokenized_datasets = dataset.map(prepare_features, batched=True, remove_columns=dataset["train"].column_names)

# Set up evaluation metrics
squad_metric = evaluate.load("squad")

def compute_metrics(predictions):
    start_logits, end_logits = predictions.predictions
    start_positions, end_positions = predictions.label_ids

    predicted_answers = []
    for i in range(len(start_logits)):
        start = np.argmax(start_logits[i])
        end = np.argmax(end_logits[i]) + 1

        predicted_answer = tokenizer.decode(tokenized_datasets["validation"]["input_ids"][i][start:end], skip_special_tokens=True)
        predicted_answers.append({"id": str(i), "prediction_text": predicted_answer})

    references = []
    for i in range(len(start_positions)):
        start_idx = start_positions[i]
        end_idx = end_positions[i] + 1

        true_answer = tokenizer.decode(tokenized_datasets["validation"]["input_ids"][i][start_idx:end_idx], skip_special_tokens=True)
        answer_start = dataset["validation"][i]["answers"]["answer_start"][0]

        references.append({"id": str(i), "answers": {"text": [true_answer], "answer_start": [answer_start]}})

    return squad_metric.compute(predictions=predicted_answers, references=references)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=[],
    logging_dir="./logs",
    save_strategy="no"
)

# Trainer for distillation
class DistilTrainer(Trainer):
    def __init__(self, teacher_model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model.to(self.args.device)

    def compute_loss(self, model, inputs, return_outputs=False):
        inputs = {k: v.to(self.args.device) for k, v in inputs.items()}
        outputs = model(**inputs)
        loss = outputs.loss

        self.teacher_model.eval()
        with torch.no_grad():
            teacher_outputs = self.teacher_model(**inputs)

        distillation_loss = (
            torch.nn.functional.kl_div(
                torch.nn.functional.log_softmax(outputs.start_logits, dim=-1),
                torch.nn.functional.softmax(teacher_outputs.start_logits, dim=-1),
                reduction="batchmean",
            )
            + torch.nn.functional.kl_div(
                torch.nn.functional.log_softmax(outputs.end_logits, dim=-1),
                torch.nn.functional.softmax(teacher_outputs.end_logits, dim=-1),
                reduction="batchmean",
            )
        ) / 2

        total_loss = loss + distillation_loss
        return (total_loss, outputs) if return_outputs else total_loss

teacher_model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

trainer = DistilTrainer(
    teacher_model=teacher_model,
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train and evaluate the model
trainer.train()
results = trainer.evaluate()
print("DistilBERT Results with distillation on SQuAD:", results)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


## BERT on SQuAD Dataset (Without Distillation)

In [ ]:
# Necessary imports remain the same
from transformers import AutoTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import evaluate

# Load SQuAD 1.1 dataset
squad = load_dataset("squad")

# Initialize evaluation metric
metric = evaluate.load("squad")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for i, offsets in enumerate(offset_mapping):
        answer = examples["answers"][i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        # Find the start and end token positions
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offsets[context_start][0] > start_char or offsets[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Apply preprocessing
encoded_dataset = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)


# Function to compute metrics for SQuAD
def compute_metrics(eval_pred):
    """Computes and returns a dictionary of metrics."""
    start_logits, end_logits = eval_pred.predictions
    start_positions, end_positions = eval_pred.label_ids

    predictions = []
    references = []

    for i in range(len(start_logits)):
        start_pred = torch.argmax(torch.tensor(start_logits[i])).item()
        end_pred = torch.argmax(torch.tensor(end_logits[i])).item()

        # Retrieve input_ids and only pass those for decoding
        input_ids = encoded_dataset["validation"][i]["input_ids"]
        prediction_text = tokenizer.decode(input_ids[start_pred:end_pred+1], skip_special_tokens=True)

        predictions.append({"id": squad["validation"][i]["id"], "prediction_text": prediction_text})
        references.append({"id": squad["validation"][i]["id"], "answers": squad["validation"][i]["answers"]})

    return metric.compute(predictions=predictions, references=references)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="no",
    logging_steps=10,
    eval_steps=10,
    report_to="none"
)

# Load DistilBERT model
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train and evaluate the model
trainer.train()
results = trainer.evaluate()
print("BERT Results without distillation on SQuAD:", results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact Match,F1
1,0.995000,1.033708,70.312204,79.837162
2,0.898000,1.007052,71.532640,81.047689
3,0.490500,1.086823,71.977294,81.229275


BERT Results without distillation on SQuAD: {'eval_loss': 1.0868231058120728, 'eval_exact_match': 71.97729422894986, 'eval_f1': 81.22927463053034, 'eval_runtime': 67.8133, 'eval_samples_per_second': 155.869, 'eval_steps_per_second': 9.747, 'epoch': 3.0}
